# Convolutional Neural Network

### Importing the libraries

In [16]:
import tensorflow as tf
from django.contrib.admin.templatetags.admin_list import results
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [17]:
tf.__version__

'2.20.0'

## Setup Tensorboard

In [18]:
from tensorflow.keras.callbacks import TensorBoard

file_name = 'my_saved_model'

tensorboard = TensorBoard(log_dir="logs\\{}".format(file_name))

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [19]:
# augments the image with zoom, mirroring, feature scaling etc.
train_datagen = ImageDataGenerator(
    # rescale: applies feature scaling for each pixel. A pixel is a number between 0-255. When you multiply this nummber for example 128 * rescale of 1/255 you get 0.5. The output number is normalized between 0 and 1.
    rescale = 1./255,
    # Bild wird verzogen. Auf wenn das Bild verzogen ist, soll das Objekt noch erkennt werden auf dem Bild. Dies ist ein Schritt für die Datenerweiterung (Augmentation). The value 0.2 means you can incline the image until 11 degree.
    shear_range = 0.2,
    # Zoom range of 0.2: it's a part of the data augmentation and will zoom in for max 20% and can zoom out for max 20%. This will help to improve the quality of the image recognition because with that the model can also recognize a object in the image when it's bigger oder smaller. E.g. the cat is in the background of the image (zoom-out) or on the image is only the head of the cat without the body (zoom-in).
    zoom_range = 0.2,
    # horizontal_flip will randomly mirror the images. E.g. on the image is a cat which look to the right. This parameter will mirror this image and afterwards the cat will look to the left. To set this parameter to true will help the model to learn that the direction if the cat looks to the left or to the right doesn't matter. The label will stay a cat.
    horizontal_flip = True
)

# connects the dataframe with the path of the training images
training_set = train_datagen.flow_from_directory(
    './dataset/training_set',
    # all images will be resized from their original size to a image of 64px width and 64px height
    target_size = (64, 64),
    batch_size = 32,
    class_mode = 'binary'
)

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [20]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory(
    './dataset/test_set',
    target_size = (64, 64),
    batch_size = 32,
    class_mode = 'binary'
)

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [21]:
# Sequential: seqzence of layers
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [22]:
# filters = 32: How many features should be detected. 32 means the model will detect 32 different features in the image (e.g. nose, mouth)
# kernel_size = 3: kernel means feature map and is in this example a 3x3 Pixels Block which is "pushed" over the image to detect where is a feature in the image
# input_shape = [64, 64, 3]: 64px widht, 64px height and 3 colors rgb. For Black-white image you whould enter 1.
cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, activation = 'relu', input_shape = [64, 64, 3]))

### Step 2 - Pooling

In [23]:
# pool_size = 2: a 2x2 Pixels Block will be used for max pooling
# strides = 2: How many Pixels the block should change. For strides of 2 the change will be 2 Pixels. E.g. for a 2x2 block the next the will be after 2 Pixels therefore it will take the next 2x2 block an not the result of the old one.
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))

### Adding a second convolutional layer

In [24]:
# Note: the input_shape parameter is only required in the first hidden layer to connect with the input layer. In this Layer, the second hidden layer, this parameter isn't needed.
cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, activation = 'relu'))
# Note: pool_size and strides should be the same number to have a smooth transition
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))

### Step 3 - Flattening

In [25]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [26]:
# this layer takes all the features of the vector from the flattening step und connects this features to the neurons in the full connection layer.
cnn.add(tf.keras.layers.Dense(units = 128, activation = 'relu'))

### Step 5 - Output Layer

In [27]:
cnn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [28]:
cnn.compile(
    # weight adjustement with gradient descent (adam -> Adaptive Moment Estimation)
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

### Training the CNN on the Training set and evaluating it on the Test set

In [29]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25, callbacks = [tensorboard])

Epoch 1/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 29s 111ms/step - accuracy: 0.5913 - loss: 0.6630 - val_accuracy: 0.6990 - val_loss: 0.5873
Epoch 2/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 37s 148ms/step - accuracy: 0.6951 - loss: 0.5800 - val_accuracy: 0.6765 - val_loss: 0.5904
Epoch 3/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 36s 144ms/step - accuracy: 0.7210 - loss: 0.5416 - val_accuracy: 0.7240 - val_loss: 0.5381
Epoch 4/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 34s 138ms/step - accuracy: 0.7404 - loss: 0.5174 - val_accuracy: 0.7495 - val_loss: 0.5085
Epoch 5/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 33s 131ms/step - accuracy: 0.7527 - loss: 0.5001 - val_accuracy: 0.7615 - val_loss: 0.4981


In [30]:
# loads the tensorboard extension
%load_ext tensorboard
# shows the path to the log file
%tensorboard --logdir logs/my_saved_model/train

## Part 4 - Making a single prediction

In [31]:
import numpy as np
from tensorflow.keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_2.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
# add extra fake dimension for the batch_size.
# axis = 0: this dimension will be on the first axis
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
# result[0][0]: first [0] is for the number of batch -> the first batch. Second [0] is for the number of element in the batch -> the first und single element
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step


In [32]:
print(prediction)

dog
